In [1]:
# graph representation in Adjacency Matrix

class Graph:
    def __init__(self, num_vertices):
        self.graph = [[0]*num_vertices for _ in range(num_vertices)]
        self.num_vertices = num_vertices
    
    def add_edge(self, u, v, weight=1):
        self.graph[u][v] = weight
        self.graph[v][u] = weight # for undirected graph

In [6]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n
    
    def find(self, u):
        if self.parent[u]!=u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]
    
    def union(self,u,v):
        root_u = self.find(u)
        root_v = self.find(v)
        if root_u != root_v:
            if self.rank[root_u] > self.rank[root_v]:
                self.parent[root_v] = root_u
            elif self.rank[root_u] < self.rank[root_v]:
                self.parent[root_u] = root_v
            else:
                self.parent[root_v] = root_u
                self.rank[root_u] += 1

In [8]:
# representation in Adjacency List:
from collections import defaultdict, deque
import heapq
class Graph:
    def __init__(self, num_vertices):
        self.graph = defaultdict(list)
        self.num_vertices = num_vertices
    
    def add_edge(self, u, v):
        self.graph[u].append(v)
        self.graph[v].append(u) # for undirected graph

    def dfs(self, start, visited = None):
        if visited is None:
            visited = set()
        visited.add(start)
        print(start, end=' ')
        for neighbor in self.graph[start]:
            if neighbor not in visited:
                self.dfs(neighbor, visited)
    
    def bfs(self, start):
        visited = set()
        queue = deque([start])
        visited.add(start)
        while queue:
            vertex = queue.popleft()
            print(vertex, end=' ')
            for neighbor in self.graph[vertex]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
    
    def topological_sort_util(self, v, visited, stack):
        visited[v] = True
        for neighbor in self.graph[v]:
            if not visited[neighbor]:
                self.topological_sort_util(neighbor, visited, stack)
        stack.insert(0,v)
    
    def topological_sort(self):
        visited = {vertex: False for vertex in self.graph}
        stack = []
        for vertex in self.graph:
            if not visited[vertex]:
                self.topological_sort_util(vertex, visited, stack)
        return stack
    
    def is_cyclic_util(self, v, visited, parent):
        visited[v] = True
        for neighbor in self.graph[v]:
            if not visited[neighbor]:
                if self.is_cyclic_util(neighbor, visited, v):
                    return True
            elif parent != neighbor:
                return True
        return False
    
    def is_cyclic(self):
        visited = [False]*self.num_vertices
        for vertex in range(self.num_vertices):
            if not visited[vertex]:
                if self.is_cyclic_util(vertex, visited, -1):
                    return True
        return False
    
    # Prims algo for Minimum spanning tree

    def prim(self, start):
        mst = []
        visited = set([start])
        edges = [(cost, start, to) for to, cost in self.graph[start].items()]
        heapq.heapify(edges)

        while edges:
            cost, frm, to = heapq.heappushpop(edges)
            if to not in visited:
                visited.add(to)
                mst.append((frm, to, cost))

                for to_next, cost in self.graph[to].items():
                    if to_next not in visited:
                        heapq.heappush(edges, (cost, to, to_next))
        return mst
    
    #krushkal's Algo

    def kruskal(self, graph):
        mst = []
        edges = sorted(graph, key=lambda item:item[2])
        ds = DisjointSet(self.num_vertices)

        for edge in edges:
            u,v, weight = edge
            if ds.find(u) != ds.find(v):
                ds.union(u,v)
                mst.append(edge)
        return mst

g = Graph(5)
g.add_edge(0, 1)
g.add_edge(0, 4)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(1, 4)
g.add_edge(2, 3)
g.add_edge(3, 4)

print("DFS Traversal:")
g.dfs(0)
print("\nBFS Traversal:")
g.bfs(0)
print("\nTopological Sort:")
print(g.topological_sort())
print("\nCycle Detection (Undirected):")
print(g.is_cyclic())

DFS Traversal:
0 1 2 3 4 
BFS Traversal:
0 1 4 2 3 
Topological Sort:
[0, 1, 2, 3, 4]

Cycle Detection (Undirected):
True
